In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llamabot import QueryBot
import git
from IPython.display import display, Markdown

# Eric Ma Q&A

This shows how to build a blog Q&A bot using the text contents of Eric Ma's blog.

## Setup: Download blog data

In [ ]:
import tempfile
from pathlib import Path

# Create a temporary directory
temp_dir = tempfile.TemporaryDirectory(dir="/tmp")


repo_url = "https://github.com/duckdb/duckdb-web"
# Clone the repository into the temporary directory
repo = git.Repo.clone_from(repo_url, temp_dir.name)

# Set the root directory to the cloned repository
root_dir = Path(temp_dir.name)

In [ ]:
from slugify import slugify
import chromadb

client = chromadb.PersistentClient(path=str(Path.home() / ".llamabot" / "chroma.db"))
collection = client.create_collection(slugify(repo_url), get_or_create=True)

results = collection.get()

In [ ]:
source_file_extensions = [
    "py",
    "jl",
    "R",
    "ipynb",
    "md",
    "tex",
    "txt",
    "lr",
    "rst",
]


source_files = []
for extension in source_file_extensions:
    files = list(root_dir.rglob(f"*.{extension}"))
    print(f"Found {len(files)} files with extension {extension}.")
    source_files.extend(files)

In [ ]:
from slugify import slugify
bot = QueryBot(
    system_prompt="You are an expert in the code repository given to you.",
    collection_name=slugify(repo_url),
    document_paths=source_files,
)

In [ ]:
bot("Give me an example of lambda functions in DuckDB.")

In [ ]:
bot("What is your view on building a digital portfolio?")

In [ ]:
bot("What were your experiences with the SciPy conference?")

In [ ]:
bot("What tutorials did you attend at the SciPy conference in 2023?")

## LlamaBot Code Query

In [ ]:
from numpy import source
from llamabot.file_finder import recursive_find
from pyprojroot import here

source_python_files = recursive_find(root_dir=here() / "llamabot", file_extension=".py")

codebot = QueryBot(
    "You are an expert in code Q&A.",
    collection_name="llamabot",
    document_paths=source_python_files,
    model_name="gpt-4-1106-preview",
)

In [ ]:
codebot("How do I find all the files in a directory?")

In [ ]:
codebot("Which Bot do I use to chat with my documents?")

In [ ]:
codebot("Explain to me the architecture of SimpleBot.")

In [ ]:
codebot("What are the CLI functions available in LlamaBot?")

In [ ]:
from llamabot.bot.qabot import DocQABot

codebot = DocQABot(
    collection_name="llamabot",
)
codebot.add_documents(document_paths=source_python_files)

In [ ]:
codebot(
    "Does LlamaBot provide a function to find all files recursively in a directory?"
)